In [1]:
import os
import torch
os.environ["HF_DATASETS_CACHE"] = "./hf_cache"
os.environ["HF_HOME"] = "./hf_cache"

ModuleNotFoundError: No module named 'torch'

In [3]:
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

model_path = "liuhaotian/llava-v1.5-7b"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path)
)

You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/gpfs/slayman/pi/gerstein/xt86/billq/cpsc477_project/cpsc477_billq/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
from llava.mm_utils import process_images
from PIL import Image
import requests
from io import BytesIO
def load_image(image_file):
    if image_file.startswith('http://') or image_file.startswith('https://'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image


In [ ]:
image = "https://storage.lilbillbiscuit.com/IMG_2184.jpeg"
image = load_image(image) # note this is a PIL image
image_size = image.size
image_tensor = process_images([image], image_processor, model.config) # 
if type(image_tensor) is list:
    image_tensor = [image.to(model.device, dtype=torch.float16) for image in image_tensor]
else:
conv    image_tensor = image_tensor.to(model.device, dtype=torch.float16)

In [ ]:
def infer_conv_mode(model_name):
    if "llama-2" in model_name.lower():
        conv_mode = "llava_llama_2"
    elif "mistral" in model_name.lower():
        conv_mode = "mistral_instruct"
    elif "v1.6-34b" in model_name.lower():
        conv_mode = "chatml_direct"
    elif "v1" in model_name.lower():
        conv_mode = "llava_v1"
    elif "mpt" in model_name.lower():
        conv_mode = "mpt"
    else:
        conv_mode = "llava_v0"
    return conv_mode

In [ ]:
inp = "Can you describe this image?"

In [ ]:
from llava.conversation import conv_templates, SeparatorStyle
conv = conv_templates[infer_conv_mode(model_path)].copy()

if image is not None:
    if model.config.mm_use_im_start_end:
        inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
    else:
        inp = DEFAULT_IMAGE_TOKEN + '\n' + inp
    conv.append_message(conv.roles[0], inp)
    image = None
else:
    conv.append_message(conv.roles[0], inp)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

In [ ]:
from llava.mm_utils import process_images, tokenizer_image_token, get_model_name_from_path
input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
# streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
with torch.inference_mode():
    output_ids = model.generate(
      input_ids, 
      images = image_tensor,
      image_sizes = [image_size],
      temperature = 0.7,
      max_new_tokens = 2048,
      use_cache = True
    )
outputs = tokenizer.decode(output_ids[0]).strip()
print(outputs)


/gpfs/slayman/pi/gerstein/xt86/billq/cpsc477_project/cpsc477_billq/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


<s> The image features a wooden door with a large wooden frame, giving it a rustic appearance. The door is open, revealing a hallway with a wooden wall on the other side. The hallway is adorned with a wooden panel, adding to the overall rustic theme.

In the background, there is a chair placed against the wall, and a dining table can be seen further back. A person is standing near the table, possibly enjoying a meal or engaging in conversation. The scene creates a warm and inviting atmosphere.</s>
